# python: types

This code is an extension of the work by Simon, which can be found here -> 
[article by Simon](https://aboutsimon.com/blog/2018/04/04/Python3-Type-Checking-And-Data-Validation-With-Type-Hints.html)

In [1]:
pip install mypy

  Using cached https://files.pythonhosted.org/packages/dd/9c/6e5db15fc89d2e8d4e436aba77a7ac0290e476cdb6100be4c99b10272593/mypy-0.761-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/03/92/705fe8aca27678e01bbdd7738173b8e7df0088a2202c80352f664630d638/typing_extensions-3.7.4.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/34/de/d0cfe2ea7ddfd8b2b8374ed2e04eeb08b6ee6e1e84081d151341bba596e5/typed_ast-1.4.0.tar.gz
  Running setup.py install for typed-ast: started
    Running setup.py install for typed-ast: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


    ERROR: Complete output from command /var/mobile/Containers/Data/Application/FF91FDAC-FD3E-4A5E-A092-87C8CADAA613/Library/bin/python3 -u -c 'import setuptools, tokenize;__file__='"'"'/private/var/mobile/Containers/Data/Application/FF91FDAC-FD3E-4A5E-A092-87C8CADAA613/tmp/pip-install-3ekkimgr/typed-ast/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /private/var/mobile/Containers/Data/Application/FF91FDAC-FD3E-4A5E-A092-87C8CADAA613/tmp/pip-record-pttf5xon/install-record.txt --single-version-externally-managed --compile:
    ERROR: running install
    running build
    running build_py
    creating build
    creating build/lib.iphoneos-10.14-iPhone12,5-3.7
    creating build/lib.iphoneos-10.14-iPhone12,5-3.7/typed_ast
    copying typed_ast/conversions.py -> build/lib.iphoneos-10.14-iPhone12,5-3.7/typed_ast
    copying typed_ast/__init__.py ->

In [2]:
import sys, re, os

from typing import get_type_hints
from functools import wraps
from inspect import getfullargspec

## validate_input()

helper function to validate the input, and return the expected output type

In [3]:
def validate_input(obj, **kwargs):
    hints = get_type_hints(obj)
    return_type = None

    # iterate all type hints
    for attr_name, attr_type in hints.items():
        if attr_name == 'return':
            return_type = attr_type
            continue

        if not isinstance(kwargs[attr_name], attr_type):
            raise TypeError('Argument %r is not of type %s' % (attr_name, attr_type))

    return return_type

## type_check()

decorator to wrap a function and enforce the types in and out

In [4]:
def type_check(decorator):
    @wraps(decorator)
    def wrapped_decorator(*args, **kwargs):
        # translate *args into **kwargs
        func_args = getfullargspec(decorator)[0]
        kwargs.update(dict(zip(func_args, args)))

        return_type = validate_input(decorator, **kwargs)
        return_value = decorator(**kwargs)
        
        if return_type and not isinstance(return_value, return_type):
            raise TypeError('return value %r is not of type %s' % (return_value, return_type))
        return return_value

    return wrapped_decorator

## addition()

test function to validate inputs

In [5]:
@type_check
def addition(number: int, other_number: int) -> int:
    return number + other_number

### test valid

In [6]:
r = addition(1, 2)
print('addition(1,2)=%d'%r)
assert(r == 3)

addition(1,2)=3


### test invalid

In [7]:
try:
    print('addition(1,\'2\')')
    print(addition(1, '2'))
except TypeError:
    e = str(sys.exc_info()[1])
    print('\t%s'%e)
    assert (e == "Argument 'other_number' is not of type <class 'int'>")

addition(1,'2')
	Argument 'other_number' is not of type <class 'int'>


## subtraction()

to test valid output return values

In [8]:
@type_check
def subtraction(number: int) -> int:
    if number == 1:
        return 1
    return 'number=%d'%number

### test valid

In [9]:
r = subtraction(1)
print('subtraction(1)=%d'%r)
assert(r == 1)

subtraction(1)=1


### test invalid

In [10]:
try:
    print('subtraction(2)')
    print(subtraction(2))
except TypeError:
    e = str(sys.exc_info()[1])
    print('\t%s'%e)
    assert (e == "return value 'number=2' is not of type <class 'int'>")

subtraction(2)
	return value 'number=2' is not of type <class 'int'>


# conclusion

usefull wrappers to encorce type definitions in python 3